In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime, timedelta

# --- Orbital Parameters ---
AU = 1.496e8  # km, astronomical unit
earth_orbit_radius = AU
moon_orbit_radius = 384400  # km
moon_orbit_scale = 25  # for visual clarity
moon_orbit_vis = moon_orbit_radius * moon_orbit_scale

# --- Date frames: every hour from 2024-04-09 00:00 to 2024-04-10 24:00 ---
start_date = datetime(2024, 4, 9, 0, 0)
end_date   = datetime(2024, 4, 10, 23, 0)
n_frames = int((end_date - start_date).total_seconds() // 3600) + 2  # +2 to include last hour as 24:00
dates = [start_date + timedelta(hours=i) for i in range(n_frames)]

# --- Functions for angles/positions ---
def date_to_earth_angle(dt):
    ve2024 = datetime(2024, 3, 20)
    days_since_ve = (dt - ve2024).days + (dt - ve2024).seconds / 86400
    angle = 2 * np.pi * days_since_ve / 365.25
    return angle

def get_positions(d):
    ea = date_to_earth_angle(d)
    ex, ey = earth_orbit_radius * np.cos(ea), earth_orbit_radius * np.sin(ea)
    moon_period = 27.321  # days
    moon_phase = 2 * np.pi * ((d - dates[0]).total_seconds() / (moon_period * 86400))
    mx, my = ex + moon_orbit_vis * np.cos(moon_phase), ey + moon_orbit_vis * np.sin(moon_phase)
    return (ex, ey, ea), (mx, my)

# --- Static figure and orbits ---
fig = plt.figure(figsize=(9, 9))
ax = plt.subplot(1, 1, 1)
ax.set_aspect('equal')
ax.set_xlim(-1.3*AU, 1.3*AU)
ax.set_ylim(-1.3*AU, 1.3*AU)
theta = np.linspace(0, 2*np.pi, 500)
ax.plot(earth_orbit_radius * np.cos(theta), earth_orbit_radius * np.sin(theta),
        '-', color='gray', lw=1, alpha=0.7, label="Earth's Orbit")

# --- Animation function ---
def update(frame):
    ax.cla()
    ax.set_aspect('equal')
    ax.set_xlim(-1.3*AU, 1.3*AU)
    ax.set_ylim(-1.3*AU, 1.3*AU)
    d = dates[frame]
    title_str = d.strftime("Sun–Earth–Moon System: %Y-%m-%d %H:%M")
    ax.set_title(title_str, fontsize=15)
    # Sun
    ax.plot(0, 0, 'o', color='orange', markersize=30, label='Sun', zorder=2)
    ax.text(0, -0.06*AU, 'Sun', color='orange', fontsize=13, ha='center', va='top')
    # Earth's orbit
    ax.plot(earth_orbit_radius * np.cos(theta), earth_orbit_radius * np.sin(theta),
            '-', color='gray', lw=1, alpha=0.7, label="Earth's Orbit")
    # Earth & Moon
    (ex, ey, ea), (mx, my) = get_positions(d)
    # Draw Moon's orbit (visual)
    moon_orbit = plt.Circle((ex, ey), moon_orbit_vis, color='purple', fill=False, linestyle='--', lw=0.8, alpha=0.5, zorder=1)
    ax.add_artist(moon_orbit)
    # Earth marker
    ax.plot(ex, ey, 'o', color='royalblue', markersize=16, zorder=3)
    ax.text(ex, ey - 0.06*AU, f"Earth", color='royalblue', fontsize=12, ha='center')
    # Moon marker
    ax.plot(mx, my, 'o', color='rebeccapurple', markersize=10, zorder=3)
    ax.text(mx, my + 0.06*AU, f"Moon", color='rebeccapurple', fontsize=12, ha='center')
    # --- Draw globe with continents at Earth's position, including axial rotation ---
    size = 0.13 * AU
    # Compute Earth's axial spin for the hour
    day_fraction = d.hour / 24.0 + d.minute / 1440.0 + d.second / 86400.0
    spin = 360.0 * day_fraction
    # The globe's central longitude: includes both orbital orientation AND daily spin
    earth_central_longitude = (-np.degrees(ea) + spin) % 360
    inset = fig.add_axes([
        0.5 + ex/(2.6*AU) - size/(2*1.3*AU),
        0.5 + ey/(2.6*AU) - size/(2*1.3*AU),
        size/(1.3*AU),
        size/(1.3*AU)
    ], projection=ccrs.Orthographic(central_longitude=earth_central_longitude, central_latitude=23.5))
    inset.patch.set_alpha(0)
    inset.set_global()
    inset.add_feature(cfeature.LAND, color='lightgreen')
    inset.add_feature(cfeature.OCEAN, color='skyblue')
    inset.add_feature(cfeature.COASTLINE, linewidth=0.5)
    inset.set_xticks([]); inset.set_yticks([])
    inset.set_title('', fontsize=7)
    # Legend
    ax.legend(loc='lower right', fontsize=11)
    ax.axis('off')
    return []

ani = animation.FuncAnimation(fig, update, frames=n_frames, interval=100, blit=False)

# --- Save as GIF ---
ani.save("sun_earth_moon_anim.gif", writer="pillow", fps=10)
print("Saved sun_earth_moon_anim.gif")

# --- For notebook inline display ---
from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 50_000_000
HTML(ani.to_jshtml())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime, timedelta

# Parameters
earth_radius_vis = 0.15   # visual earth radius (arbitrary units)
moon_distance_vis = 1.0   # arbitrary units for display
moon_radius_vis = 0.05    # visual radius of moon
sun_offset = -2.0         # sun at x=-2, y=0 (arbitrary units)
frame_interval = 100      # ms between frames

# Dates: every hour April 9, 2024 00:00 to April 10, 2024 24:00
start_date = datetime(2024, 4, 9, 0, 0)
end_date   = datetime(2024, 4, 10, 23, 0)
n_frames = int((end_date - start_date).total_seconds() // 3600) + 2
dates = [start_date + timedelta(hours=i) for i in range(n_frames)]

# Moon's sidereal period
moon_period_days = 27.321

def get_moon_pos(d):
    """Returns moon angle and coordinates (in Earth-centered frame, right = 0°)"""
    hours_since_start = (d - dates[0]).total_seconds() / 3600.0
    phase = 2 * np.pi * (hours_since_start / (moon_period_days * 24))
    mx = moon_distance_vis * np.cos(phase)
    my = moon_distance_vis * np.sin(phase)
    return mx, my

def get_earth_rotation_angle(d):
    # Earth's daily rotation + orbital orientation (tiny shift for 2 days)
    # We'll use daily spin only for globe rotation here
    frac_day = d.hour / 24.0 + d.minute / 1440.0 + d.second / 86400.0
    return (360.0 * frac_day) % 360

fig, axes = plt.subplots(1, 2, figsize=(11, 6))
ax0, ax1 = axes
plt.tight_layout(pad=3)

def update(frame):
    for ax in axes: ax.cla()
    d = dates[frame]

    # --- Left panel: Sun at 9 o'clock, sun vector, Earth position ---
    ax0.set_xlim(-2.5, 2.5)
    ax0.set_ylim(-1.5, 1.5)
    ax0.axis('off')
    # Draw sun at left
    ax0.plot(sun_offset, 0, marker='o', markersize=30, color='orange', zorder=3)
    ax0.text(sun_offset, -0.7, 'Sun', fontsize=13, ha='center', va='top', color='orange')
    # Sun vector (rightwards)
    ax0.arrow(sun_offset+0.3, 0, 1.7, 0, head_width=0.08, head_length=0.18, fc='gold', ec='gold', length_includes_head=True, lw=2, zorder=2)
    ax0.text(0, 0.15, "Sunlight", fontsize=11, ha='center', va='bottom', color='gold')
    # Earth outline at (0,0)
    earth_circle = plt.Circle((0, 0), earth_radius_vis, ec='royalblue', lw=2, fill=False, zorder=1)
    ax0.add_artist(earth_circle)
    ax0.text(0, -earth_radius_vis-0.1, 'Earth', fontsize=12, ha='center', va='top', color='royalblue')
    # Moon position (gray circle)
    mx, my = get_moon_pos(d)
    moon_circle = plt.Circle((mx, my), moon_radius_vis, color='#cccccc', ec='gray', lw=1.2, zorder=2)
    ax0.add_artist(moon_circle)
    ax0.text(mx, my+0.11, "Moon", ha='center', va='bottom', fontsize=10, color='dimgray')
    # Arrow from Earth to Moon
    ax0.arrow(0, 0, mx*0.85, my*0.85, head_width=0.04, head_length=0.07, fc='gray', ec='gray', lw=1, zorder=1, length_includes_head=True, alpha=0.5)
    # Orbit line for moon
    theta = np.linspace(0, 2*np.pi, 200)
    ax0.plot(moon_distance_vis*np.cos(theta), moon_distance_vis*np.sin(theta), '--', color='gray', lw=0.7, alpha=0.4)

    # --- Right panel: Rotating Earth globe and Moon position ---
    ax1.set_xlim(-0.4, 0.6)
    ax1.set_ylim(-0.4, 0.4)
    ax1.axis('off')
    ax1.text(0.22, 0.35, d.strftime('%Y-%m-%d %H:%M'), fontsize=13, ha='center', va='center')
    # Rotating globe (with continents)
    size = 0.28
    earth_rotation = get_earth_rotation_angle(d)
    globe_ax = fig.add_axes([
        0.59, 0.23, size, size
    ], projection=ccrs.Orthographic(central_longitude=earth_rotation, central_latitude=23.5), zorder=3)
    globe_ax.patch.set_alpha(0)
    globe_ax.set_global()
    globe_ax.add_feature(cfeature.LAND, color='lightgreen')
    globe_ax.add_feature(cfeature.OCEAN, color='skyblue')
    globe_ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    globe_ax.set_xticks([]); globe_ax.set_yticks([])
    globe_ax.set_title('', fontsize=7)
    # Draw Earth's limb on ax1 for visual reference
    limb = plt.Circle((0,0), earth_radius_vis, ec='royalblue', fill=False, lw=2, zorder=2)
    ax1.add_artist(limb)
    # Draw Moon on ax1 (in correct place relative to globe)
    ax1.plot(mx, my, 'o', markersize=23, color='#cccccc', mec='gray', mew=1.7, zorder=4)
    ax1.text(mx, my+0.13, "Moon", ha='center', va='bottom', fontsize=11, color='dimgray')
    # Moon orbit
    ax1.plot(moon_distance_vis*np.cos(theta), moon_distance_vis*np.sin(theta), '--', color='gray', lw=0.8, alpha=0.5)

    # Add Sun vector (faint, for reference)
    ax1.arrow(-0.25, 0, 0.37, 0, head_width=0.05, head_length=0.07, fc='gold', ec='gold', lw=2, alpha=0.3, length_includes_head=True, zorder=1)
    ax1.text(-0.22, -0.09, "Sunlight", fontsize=10, color='gold')

    # Remove globe axis after draw, or Cartopy will leak axes
    plt.draw()
    globe_ax.remove()

ani = animation.FuncAnimation(fig, update, frames=n_frames, interval=frame_interval, blit=False)

ani.save("sun_earth_moon_sideby.gif", writer="pillow", fps=10)
print("Saved sun_earth_moon_sideby.gif")

# For notebook inline display
from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 50_000_000
HTML(ani.to_jshtml())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from datetime import datetime, timedelta

# Parameters
earth_radius_vis = 0.15   # visual earth radius (arbitrary units)
moon_distance_vis = 1.0   # arbitrary units for display
moon_radius_vis = 0.05    # visual radius of moon
sun_offset = -2.0         # sun at x=-2, y=0 (arbitrary units)
frame_interval = 100      # ms between frames

# Dates: every hour April 9, 2024 00:00 to April 10, 2024 24:00
start_date = datetime(2024, 4, 9, 0, 0)
end_date   = datetime(2024, 4, 10, 23, 0)
n_frames = int((end_date - start_date).total_seconds() // 3600) + 2
dates = [start_date + timedelta(hours=i) for i in range(n_frames)]

moon_period_days = 27.321

def get_moon_pos(d):
    """Returns moon angle and coordinates (in Earth-centered frame, right = 0°)"""
    hours_since_start = (d - dates[0]).total_seconds() / 3600.0
    phase = 2 * np.pi * (hours_since_start / (moon_period_days * 24))
    mx = moon_distance_vis * np.cos(phase)
    my = moon_distance_vis * np.sin(phase)
    return mx, my

fig, ax0 = plt.subplots(figsize=(7, 6))
plt.tight_layout(pad=3)

def update(frame):
    ax0.cla()
    d = dates[frame]
    # --- Left panel: Sun at 9 o'clock, sun vector, Earth position ---
    ax0.set_xlim(-2.5, 2.5)
    ax0.set_ylim(-1.5, 1.5)
    ax0.axis('off')
    ax0.set_title(d.strftime("Sun–Earth–Moon: %Y-%m-%d %H:%M"), fontsize=14)
    # Draw sun at left
    ax0.plot(sun_offset, 0, marker='o', markersize=30, color='orange', zorder=3)
    ax0.text(sun_offset, -0.7, 'Sun', fontsize=13, ha='center', va='top', color='orange')
    # Sun vector (rightwards)
    ax0.arrow(sun_offset+0.3, 0, 1.7, 0, head_width=0.08, head_length=0.18, fc='gold', ec='gold', length_includes_head=True, lw=2, zorder=2)
    ax0.text(0, 0.15, "Sunlight", fontsize=11, ha='center', va='bottom', color='gold')
    # Earth outline at (0,0)
    earth_circle = plt.Circle((0, 0), earth_radius_vis, ec='royalblue', lw=2, fill=False, zorder=1)
    ax0.add_artist(earth_circle)
    ax0.text(0, -earth_radius_vis-0.1, 'Earth', fontsize=12, ha='center', va='top', color='royalblue')
    # Moon position (gray circle)
    mx, my = get_moon_pos(d)
    moon_circle = plt.Circle((mx, my), moon_radius_vis, color='#cccccc', ec='gray', lw=1.2, zorder=2)
    ax0.add_artist(moon_circle)
    ax0.text(mx, my+0.11, "Moon", ha='center', va='bottom', fontsize=10, color='dimgray')
    # Arrow from Earth to Moon
    ax0.arrow(0, 0, mx*0.85, my*0.85, head_width=0.04, head_length=0.07, fc='gray', ec='gray', lw=1, zorder=1, length_includes_head=True, alpha=0.5)
    # Orbit line for moon
    theta = np.linspace(0, 2*np.pi, 200)
    ax0.plot(moon_distance_vis*np.cos(theta), moon_distance_vis*np.sin(theta), '--', color='gray', lw=0.7, alpha=0.4)

ani = animation.FuncAnimation(fig, update, frames=n_frames, interval=frame_interval, blit=False)

ani.save("sun_earth_moon_panel.gif", writer="pillow", fps=10)
print("Saved sun_earth_moon_panel.gif")

# For notebook inline display
from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 50_000_000
HTML(ani.to_jshtml())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime, timedelta

# Parameters
earth_radius_vis = 0.15   # schematic earth radius (axis units)
moon_distance_vis = 1.0   # arbitrary units for display
moon_radius_vis = 0.05    # visual radius of moon
sun_offset = -2.0         # sun at x=-2, y=0 (arbitrary units)
frame_interval = 100      # ms between frames

# Dates: every hour April 9, 2024 00:00 to April 10, 2024 24:00
start_date = datetime(2024, 4, 9, 0, 0)
end_date   = datetime(2024, 4, 10, 23, 0)
n_frames = int((end_date - start_date).total_seconds() // 3600) + 2
dates = [start_date + timedelta(hours=i) for i in range(n_frames)]
moon_period_days = 27.321

def get_moon_pos(d):
    hours_since_start = (d - dates[0]).total_seconds() / 3600.0
    phase = 2 * np.pi * (hours_since_start / (moon_period_days * 24))
    mx = moon_distance_vis * np.cos(phase)
    my = moon_distance_vis * np.sin(phase)
    return mx, my

def get_earth_rotation_angle(d):
    frac_day = d.hour / 24.0 + d.minute / 1440.0 + d.second / 86400.0
    return (360.0 * frac_day) % 360

# --- Set up the figure and axes ---
fig = plt.figure(figsize=(7, 7))
ax0 = fig.add_axes([0, 0, 1, 1])  # full figure for schematic
ax0.set_xlim(-2.5, 2.5)
ax0.set_ylim(-1.5, 1.5)
ax0.axis('off')

# --- Create the Cartopy globe ONCE, centered, in a fixed position ---
globe_size_frac = 0.23  # fraction of figure (adjust for size)
globe_ax = fig.add_axes([0.5 - globe_size_frac/2, 0.5 - globe_size_frac/2,
                         globe_size_frac, globe_size_frac],
                        projection=ccrs.Orthographic(central_longitude=0, central_latitude=23.5), zorder=10)
globe_ax.set_global()
globe_land = globe_ax.add_feature(cfeature.LAND, color='lightgreen')
globe_ocean = globe_ax.add_feature(cfeature.OCEAN, color='skyblue')
globe_coast = globe_ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
globe_ax.set_xticks([]); globe_ax.set_yticks([])
globe_ax.patch.set_alpha(0)
globe_ax.set_title('', fontsize=7)

def update(frame):
    ax0.cla()
    d = dates[frame]
    ax0.set_xlim(-2.5, 2.5)
    ax0.set_ylim(-1.5, 1.5)
    ax0.axis('off')
    ax0.set_title(d.strftime("Sun–Earth–Moon: %Y-%m-%d %H:%M"), fontsize=14)
    # Sun at left
    ax0.plot(sun_offset, 0, marker='o', markersize=30, color='orange', zorder=3)
    ax0.text(sun_offset, -0.7, 'Sun', fontsize=13, ha='center', va='top', color='orange')
    # Sun vector (rightwards)
    ax0.arrow(sun_offset+0.3, 0, 1.7, 0, head_width=0.08, head_length=0.18, fc='gold', ec='gold',
              length_includes_head=True, lw=2, zorder=2)
    ax0.text(0, 0.15, "Sunlight", fontsize=11, ha='center', va='bottom', color='gold')
    # Moon position (gray circle)
    mx, my = get_moon_pos(d)
    moon_circle = plt.Circle((mx, my), moon_radius_vis, color='#cccccc', ec='gray', lw=1.2, zorder=2)
    ax0.add_artist(moon_circle)
    ax0.text(mx, my+0.11, "Moon", ha='center', va='bottom', fontsize=10, color='dimgray')
    # Arrow from Earth to Moon
    ax0.arrow(0, 0, mx*0.85, my*0.85, head_width=0.04, head_length=0.07, fc='gray', ec='gray',
              lw=1, zorder=1, length_includes_head=True, alpha=0.5)
    # Orbit line for moon
    theta = np.linspace(0, 2*np.pi, 200)
    ax0.plot(moon_distance_vis*np.cos(theta), moon_distance_vis*np.sin(theta),
             '--', color='gray', lw=0.7, alpha=0.4)
    # --- Update globe rotation ---
    earth_rotation = get_earth_rotation_angle(d)
    globe_ax.cla()  # clear, but do not remove!
    globe_ax.set_global()
    globe_ax.add_feature(cfeature.LAND, color='lightgreen')
    globe_ax.add_feature(cfeature.OCEAN, color='skyblue')
    globe_ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    globe_ax.set_xticks([]); globe_ax.set_yticks([])
    globe_ax.patch.set_alpha(0)
    globe_ax.set_title('', fontsize=7)
    globe_ax.projection = ccrs.Orthographic(central_longitude=earth_rotation, central_latitude=23.5)

ani = animation.FuncAnimation(fig, update, frames=n_frames, interval=frame_interval, blit=False)

ani.save("sun_earth_moon_panel_globe.gif", writer="pillow", fps=10)
print("Saved sun_earth_moon_panel_globe.gif")

# For notebook inline display
from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 50_000_000
HTML(ani.to_jshtml())


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# --- Load Data ---
df = pd.read_parquet("combined_ionex_vtec.parquet")
if not isinstance(df.columns, pd.MultiIndex):
    raise RuntimeError("Expected MultiIndex columns for [lat, lon]!")
lats = sorted(set(lat for lat, _ in df.columns), reverse=True)
lons = sorted(set(lon for _, lon in df.columns))
all_epochs = pd.to_datetime(df.index)
all_grids = [df.loc[epoch].values.reshape((len(lats), len(lons))) for epoch in all_epochs]
vmin = np.nanmin([np.nanmin(g) for g in all_grids])
vmax = np.nanmax([np.nanmax(g) for g in all_grids])

def get_subsolar_lon(dt):
    seconds = dt.hour * 3600 + dt.minute * 60 + dt.second
    frac_of_day = seconds / 86400.0
    return (frac_of_day * 360.0) - 180.0

# --- Orbital schematic parameters ---
earth_radius_vis = 0.15
moon_distance_vis = 1.0
moon_radius_vis = 0.05
sun_offset = -2.0
moon_period_days = 27.321

def get_moon_pos(dt, start_dt):
    hours_since_start = (dt - start_dt).total_seconds() / 3600.0
    phase = 2 * np.pi * (hours_since_start / (moon_period_days * 24))
    mx = moon_distance_vis * np.cos(phase)
    my = moon_distance_vis * np.sin(phase)
    return mx, my

fig = plt.figure(figsize=(10, 8))
ax0 = fig.add_axes([0, 0, 1, 1])
ax0.set_xlim(-2.5, 2.5)
ax0.set_ylim(-1.5, 1.5)
ax0.axis('off')

globe_size_frac = 0.28
globe_ax = fig.add_axes([0.5 - globe_size_frac/2, 0.5 - globe_size_frac/2, globe_size_frac, globe_size_frac],
                        projection=ccrs.Orthographic(central_longitude=0, central_latitude=0),
                        zorder=10)
globe_ax.set_global()
globe_ax.set_xticks([]); globe_ax.set_yticks([])
globe_ax.patch.set_alpha(0)
globe_ax.set_title('', fontsize=7)

def update(frame):
    dt = pd.to_datetime(all_epochs[frame])
    ax0.cla()
    ax0.set_xlim(-2.5, 2.5)
    ax0.set_ylim(-1.5, 1.5)
    ax0.axis('off')
    ax0.set_title(dt.strftime("Sun–Earth–Moon & VTEC: %Y-%m-%d %H:%M UTC"), fontsize=14)
    # Sun at left
    ax0.plot(sun_offset, 0, marker='o', markersize=30, color='orange', zorder=3)
    ax0.text(sun_offset, -0.7, 'Sun', fontsize=13, ha='center', va='top', color='orange')
    ax0.arrow(sun_offset+0.3, 0, 1.7, 0, head_width=0.08, head_length=0.18,
              fc='gold', ec='gold', length_includes_head=True, lw=2, zorder=2)
    ax0.text(0, 0.15, "Sunlight", fontsize=11, ha='center', va='bottom', color='gold')
    # Moon
    mx, my = get_moon_pos(dt, pd.to_datetime(all_epochs[0]))
    moon_circle = plt.Circle((mx, my), moon_radius_vis, color='#cccccc', ec='gray', lw=1.2, zorder=2)
    ax0.add_artist(moon_circle)
    ax0.text(mx, my+0.11, "Moon", ha='center', va='bottom', fontsize=10, color='dimgray')
    ax0.arrow(0, 0, mx*0.85, my*0.85, head_width=0.04, head_length=0.07,
              fc='gray', ec='gray', lw=1, zorder=1, length_includes_head=True, alpha=0.5)
    theta = np.linspace(0, 2*np.pi, 200)
    ax0.plot(moon_distance_vis*np.cos(theta), moon_distance_vis*np.sin(theta),
             '--', color='gray', lw=0.7, alpha=0.4)
    # --- Draw VTEC on the globe ---
    grid = all_grids[frame]
    subsolar_lon = get_subsolar_lon(dt)
    globe_ax.cla()
    globe_ax.set_global()
    globe_ax.projection = ccrs.Orthographic(central_longitude=-subsolar_lon, central_latitude=0)
    vtec_plot = globe_ax.pcolormesh(
        lons, lats, grid, transform=ccrs.PlateCarree(), shading='auto', vmin=vmin, vmax=vmax, cmap='gist_ncar'
    )
    globe_ax.add_feature(cfeature.COASTLINE, linewidth=0.7)
    globe_ax.set_xticks([]); globe_ax.set_yticks([])
    globe_ax.patch.set_alpha(0)
    globe_ax.set_title('', fontsize=7)
    # Colorbar (only once)
    if frame == 0:
        cax = fig.add_axes([0.82, 0.37, 0.025, 0.22])
        cb = plt.colorbar(vtec_plot, cax=cax, orientation='vertical', label='VTEC (TECU)')
        cax.yaxis.set_ticks_position('right')
        cax.yaxis.set_label_position('right')

ani = animation.FuncAnimation(fig, update, frames=len(all_epochs), interval=200, blit=False)
ani.save("sun_earth_moon_vtec_single.gif", writer="pillow", fps=10)
print("Saved sun_earth_moon_vtec_single.gif")

# For notebook inline display
from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 50_000_000
HTML(ani.to_jshtml())


In [ ]:
#i think the moon is also dragging things!???

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# --- Load Data ---
df = pd.read_parquet("combined_ionex_vtec.parquet")
if not isinstance(df.columns, pd.MultiIndex):
    raise RuntimeError("Expected MultiIndex columns for [lat, lon]!")
lats = sorted(set(lat for lat, _ in df.columns), reverse=True)
lons = sorted(set(lon for _, lon in df.columns))
all_epochs = pd.to_datetime(df.index)
all_grids = [df.loc[epoch].values.reshape((len(lats), len(lons))) for epoch in all_epochs]
vmin = np.nanmin([np.nanmin(g) for g in all_grids])
vmax = np.nanmax([np.nanmax(g) for g in all_grids])

def get_subsolar_lon(dt):
    seconds = dt.hour * 3600 + dt.minute * 60 + dt.second
    frac_of_day = seconds / 86400.0
    return (frac_of_day * 360.0) - 180.0

# Orbital schematic parameters
earth_radius_vis = 0.15
moon_distance_vis = 1.0
moon_radius_vis = 0.05
sun_offset = -2.0
moon_period_days = 27.321

def get_moon_pos(dt, start_dt):
    hours_since_start = (dt - start_dt).total_seconds() / 3600.0
    phase = 2 * np.pi * (hours_since_start / (moon_period_days * 24))
    mx = moon_distance_vis * np.cos(phase)
    my = moon_distance_vis * np.sin(phase)
    return mx, my

fig = plt.figure(figsize=(10, 8))
ax0 = fig.add_axes([0, 0, 1, 1])
ax0.set_xlim(-2.5, 2.5)
ax0.set_ylim(-1.5, 1.5)
ax0.axis('off')

# Globe inset (centered)
globe_size_frac = 0.28
globe_ax = fig.add_axes([0.5 - globe_size_frac/2, 0.5 - globe_size_frac/2, globe_size_frac, globe_size_frac],
                        projection=ccrs.Orthographic(central_longitude=0, central_latitude=90),  # North Pole up
                        zorder=10)
globe_ax.set_global()
globe_ax.set_xticks([]); globe_ax.set_yticks([])
globe_ax.patch.set_alpha(0)
globe_ax.set_title('', fontsize=7)

def update(frame):
    dt = pd.to_datetime(all_epochs[frame])
    ax0.cla()
    ax0.set_xlim(-2.5, 2.5)
    ax0.set_ylim(-1.5, 1.5)
    ax0.axis('off')
    ax0.set_title(dt.strftime("Top-Down Orbit: Sun–Earth–Moon & VTEC: %Y-%m-%d %H:%M UTC"), fontsize=14)
    # Sun at left
    ax0.plot(sun_offset, 0, marker='o', markersize=30, color='orange', zorder=3)
    ax0.text(sun_offset, -0.7, 'Sun', fontsize=13, ha='center', va='top', color='orange')
    ax0.arrow(sun_offset+0.3, 0, 1.7, 0, head_width=0.08, head_length=0.18,
              fc='gold', ec='gold', length_includes_head=True, lw=2, zorder=2)
    ax0.text(0, 0.2, "Sunlight", fontsize=11, ha='center', va='bottom', color='gold')
    # Moon
    mx, my = get_moon_pos(dt, pd.to_datetime(all_epochs[0]))
    moon_circle = plt.Circle((mx, my), moon_radius_vis, color='#cccccc', ec='gray', lw=1.2, zorder=2)
    ax0.add_artist(moon_circle)
    ax0.text(mx, my+0.13, "Moon", ha='center', va='bottom', fontsize=10, color='dimgray')
    ax0.arrow(0, 0, mx*0.85, my*0.85, head_width=0.04, head_length=0.07,
              fc='gray', ec='gray', lw=1, zorder=1, length_includes_head=True, alpha=0.5)
    theta = np.linspace(0, 2*np.pi, 200)
    ax0.plot(moon_distance_vis*np.cos(theta), moon_distance_vis*np.sin(theta),
             '--', color='gray', lw=0.7, alpha=0.4)
    # --- Draw VTEC on the globe ---
    grid = all_grids[frame]
    subsolar_lon = get_subsolar_lon(dt)
    # For top-down orbital view: North Pole up, subsolar point at 9 o'clock (left) = central_longitude = -subsolar_lon + 90
    globe_ax.cla()
    globe_ax.set_global()
    globe_ax.projection = ccrs.Orthographic(central_longitude=90 - subsolar_lon, central_latitude=90)
    vtec_plot = globe_ax.pcolormesh(
        lons, lats, grid, transform=ccrs.PlateCarree(), shading='auto', vmin=vmin, vmax=vmax, cmap='gist_ncar'
    )
    globe_ax.add_feature(cfeature.COASTLINE, linewidth=0.7)
    globe_ax.set_xticks([]); globe_ax.set_yticks([])
    globe_ax.patch.set_alpha(0)
    globe_ax.set_title('', fontsize=7)
    # Colorbar (only once)
    if frame == 0:
        cax = fig.add_axes([0.82, 0.37, 0.025, 0.22])
        cb = plt.colorbar(vtec_plot, cax=cax, orientation='vertical', label='VTEC (TECU)')
        cax.yaxis.set_ticks_position('right')
        cax.yaxis.set_label_position('right')

ani = animation.FuncAnimation(fig, update, frames=len(all_epochs), interval=200, blit=False)
ani.save("topdown_sunlocked_orbit_vtec.gif", writer="pillow", fps=10)
print("Saved topdown_sunlocked_orbit_vtec.gif")

# For notebook inline display
from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 50_000_000
HTML(ani.to_jshtml())
